# Customer Segmentation with RFM
---
- **R**ecency, **F**requency, **M**onetary; metriklerinin baş harflerinin bir araya getirilmesi ile ifade edilir.
- **RFM analizi**, müşteri segmentasyonu için kullanılan bir tekniktir.
- Müşterilerin satın alma alışkanlıkları üzerinden gruplara ayrılması ve bu gruplar özelinde stratejiler geliştirilmesini sağlar.
- CRM çalışmaları için birçok başlıkta _**veriye dayalı aksiyon alma**_ imkanı sağlar.
---
- **RFM Metrikleri**
    - **Recency**: Müşterinin bizden en son ne zaman alışveriş yaptığı durumunu ifade eder.
    - **Frequency**: Müşterinin yapmış olduğu toplam işlem sayısı. (Satın alma sıklığı)
    - **Monetary**: Müşterilerin bizlere bıraktığı parasal değer.
---
- **RFM Skoru**
    - RFM metriklerini standartlaştırıp bir araya getirerek RFM skoru oluşturuyoruz.
    - _Peki neden?_; Metrikler kendi hallerinde bizlere çok fazla bir anlam ifade etmiyor. Segmentasyon yapamıyoruz. Bu yüzden metrikleri skora çevirip segmentasyon işlemini gerçekleştirebiliyoruz.

### Uçtan Uca RFM ile Müşteri Segmentasyonu
---
1. İş Problemi (Business Problem)
2. Veriyi Anlama (Data Understanding)
3. Veri Hazırlama (Data Preparation)
4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)
6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)
7. Tüm Sürecin Fonksiyonlaştırılması

#### 1. İş Problemi (Business Problem)
- Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor.


- Veri Seti Hikayesi
- https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

- Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.


- **Değişkenler**

    - **InvoiceNo**: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
    - **StockCode**: Ürün kodu. Her bir ürün için eşsiz numara.
    - **Description**: Ürün ismi
    - **Quantity**: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
    - **InvoiceDate**: Fatura tarihi ve zamanı.
    - **UnitPrice**: Ürün fiyatı (Sterlin cinsinden)
    - **CustomerID**: Eşsiz müşteri numarası
    - **Country**: Ülke ismi. Müşterinin yaşadığı ülke.

#### 2. Veriyi Anlama (Data Understanding)

In [1]:
import datetime as dt
import pandas as pd
# .py script için yapılan gösterim ayarları
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.5f" % x)

In [2]:
df_ = pd.read_excel("../datasets/online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = df_.copy()

- *df_.copy()* kullanmamın nedeni veri seti boyutu büyük olduğundan dolayı veri okuma işlemi uzun sürebilmektedir. çalışmamın ilerleyen zamanlarında tekrardan veriyi okutma ihtiyacı duyarsam bu süreyi optimize etmek, beklememek adına kullanıyorum.

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [4]:
df.shape # 525461 gözlem, 8 değişken

(525461, 8)

In [5]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

- kaç adet eşsiz ürün sayısı var?;

In [6]:
df["Description"].nunique()

4681

- bu bulunan 4681 eşsiz ürünün hangileri kaç defa faturaya yansımıştır?;

In [7]:
df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
Name: Description, dtype: int64

- en çok sipariş edilen ürün hangisidir?; (hangi üründen toplam ne kadar sipariş edildi?)

In [10]:
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


- kaç fatura kesilmiş?

In [11]:
df["Invoice"].nunique()

28816

- fatura başına toplam kaç para kazanılmıştır?

In [12]:
df["TotalPrice"] = df["Quantity"] * df["Price"] # ürünlere ödenen toplam fiyat

In [13]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom,83.40000
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom,81.00000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom,100.80000
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom,30.00000


In [14]:
df.groupby("Invoice").agg({"TotalPrice": "sum"}).head()

,TotalPrice
Invoice,
489434,505.30000
489435,145.80000
489436,630.33000
489437,310.75000
489438,2286.24000


#### 3. Veri Hazırlama (Data Preparation)

In [15]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
TotalPrice          0
dtype: int64

- *Customer ID* değişkeninde eksik değerler bulunmakta. Müşteri bazlı segmentasyon yapacağım için bu eksik değerli veri setinden çıkarmam gerekiyor.

In [16]:
df.dropna(inplace=True)

In [18]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
TotalPrice     0
dtype: int64

- veri seti hikayesinde bahsedildiği gibi iade faturaları da veri setinden çıkarmamız gerekiyor. verinin yapısını bozuyor.

In [19]:
df = df[~df["Invoice"].str.contains("C", na=False)]

#### 4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
- Recency, Frequency, Monetary


- RFM metrikleri hesaplanmaya başlandığında ilk yapmamız gereken bir analiz tarihi belirlemek. veri seti eski bir veri seti olduğunu ve tarihi one göre belirlememiz gerektiğini unutmayalım.

In [20]:
df["InvoiceDate"].max() # en son ne zaman işlem gerçekleşmiş?

Timestamp('2010-12-09 20:01:00')

In [22]:
today_date = dt.datetime(2010, 12, 11) # analiz tarihini belirledik.

In [23]:
type(today_date)

datetime.datetime

In [24]:
rfm = df.groupby("Customer ID").agg({"InvoiceDate": lambda date: (today_date - date.max()).days,
                                    "Invoice": lambda num: num.nunique(),
                                    "TotalPrice": lambda TotalPrice: TotalPrice.sum()})

- kullanıcı id'leri kırılımında gerçekleştirdiğim bu işlemde *InvoiceDate*'e uyguladığım aksiyon; belirlediğim analiz tarihinden müşterilerin en son alışveriş yaptığı tarihi çıkartıp bunu gün cinsinden ifade etmesini sağladım.

In [25]:
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.00000,165,11,372.86000
12347.00000,3,2,1323.32000
12348.00000,74,1,222.16000
12349.00000,43,3,2671.14000
12351.00000,11,1,300.93000


In [26]:
rfm.columns = ["recency", "frequency", "monetary"] # isimleri düzeltiyorum.

In [27]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4314.00000,91.26982,96.94430,1.00000,18.00000,53.00000,136.00000,374.00000
frequency,4314.00000,4.45410,8.16866,1.00000,1.00000,2.00000,5.00000,205.00000
monetary,4314.00000,2047.28866,8912.52324,0.00000,307.95000,705.55000,1722.80250,349164.35000


In [28]:
rfm = rfm[rfm["monetary"] > 0]
# monetary değeri min 0 olduğu için ve bunu istemediğimiz için düzenliyoruz

In [29]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.00000,91.17254,96.86146,1.00000,18.00000,53.00000,136.00000,374.00000
frequency,4312.00000,4.45571,8.17021,1.00000,1.00000,2.00000,5.00000,205.00000
monetary,4312.00000,2048.23824,8914.48128,2.95000,307.98750,706.02000,1723.14250,349164.35000


#### 5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)
- burada dikkat etmemiz gereken bir nokta var. frequency ve monetary değeri büyük olan müşterileri skorladığımızda 5 olarak değerlendirebiliriz. fakat recency büyük olan müşterileri skorladığımız da 1 vermeliyiz. çünkü recency yenilik olduğu için yeni işlemleri tercih ederiz. kafa karışıklığı yaratmaması adına.

In [30]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])

- *qcut* fonksiyonu verilen değişkenleri küçükten büyüğe sıralar ve çeyreklik değerleri *labels* argümanına girdiğimiz değerlerle yeniden adlandırır. 

In [31]:
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])

In [32]:
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

- *rank* metodunu kullanmamızdaki amaç; qcut ile çeyrekliklere ayırdığımızda çeyrekliklere hep aynı değerler düşmüş olduğundan valueerror döndürecek. o yüzden ilk gördüğünü ilk sınıfa ata demiş oluyoruz.

In [33]:
rfm.head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score
Customer ID,,,,,,
12346.00000,165,11,372.86000,2,2,5
12347.00000,3,2,1323.32000,5,4,2
12348.00000,74,1,222.16000,2,1,1
12349.00000,43,3,2671.14000,3,5,3
12351.00000,11,1,300.93000,5,2,1


- R ve F değerli yeterli olacağından R ve F değerlerini bir araya getirerek yazmamız gerekmektedir.

In [34]:
rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

In [35]:
rfm.head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE
Customer ID,,,,,,,
12346.00000,165,11,372.86000,2,2,5,25
12347.00000,3,2,1323.32000,5,4,2,52
12348.00000,74,1,222.16000,2,1,1,21
12349.00000,43,3,2671.14000,3,5,3,33
12351.00000,11,1,300.93000,5,2,1,51


- aslında şimdilik basit bir şekilde müşteri segmentasyonunu gerçekleştirmiş olduk. *nasıl?* bizim için en değerli müşterileri görmek istersek eğer;

In [38]:
rfm[rfm["RFM_SCORE"] == "55"].head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE
Customer ID,,,,,,,
12415.00000,11,7,19543.84000,5,5,5,55
12431.00000,9,13,4370.52000,5,5,5,55
12471.00000,10,49,20139.74000,5,5,5,55
12472.00000,5,13,11308.48000,5,5,5,55
12474.00000,14,13,5048.66000,5,5,5,55


- görece az değerli müşteriler;

In [39]:
rfm[rfm["RFM_SCORE"] == "11"].head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE
Customer ID,,,,,,,
12355.00000,203,1,488.21000,1,2,1,11
12362.00000,374,1,130.00000,1,1,1,11
12366.00000,269,1,500.24000,1,2,1,11
12368.00000,264,1,917.70000,1,3,1,11
12378.00000,198,1,1407.70000,1,4,1,11


#### 6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)
- tamam ama hep böyle tek tek yazarak mı bulacağız? 55 bana çok mantıksız geldi; o gruba bir isim mi versek? daha anlaşılır olur hem?
- segmentleri isimlendirmemiz gerekiyor..

In [40]:
# RFM isimlendirmesi
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [41]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

In [42]:
rfm.head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.00000,165,11,372.86000,2,2,5,25,cant_loose
12347.00000,3,2,1323.32000,5,4,2,52,potential_loyalists
12348.00000,74,1,222.16000,2,1,1,21,hibernating
12349.00000,43,3,2671.14000,3,5,3,33,need_attention
12351.00000,11,1,300.93000,5,2,1,51,new_customers


- peki bu segmentlerin analizini yapmak istersek?

In [43]:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

recency       frequency         monetary      
                         mean count      mean count       mean count
segment                                                             
about_to_sleep       53.81924   343   1.20117   343  441.32000   343
at_Risk             152.15876   611   3.07365   611 1188.87832   611
cant_loose          124.11688    77   9.11688    77 4099.45000    77
champions             7.11916   663  12.55354   663 6852.26417   663
hibernating         213.88571  1015   1.12611  1015  403.97784  1015
loyal_customers      36.28706   742   6.83019   742 2746.06735   742
need_attention       53.26570   207   2.44928   207 1060.35700   207
new_customers         8.58000    50   1.00000    50  386.19920    50
potential_loyalists  18.79304   517   2.01741   517  729.51099   517
promising            25.74713    87   1.00000    87  367.08678    87

- peki bu müşterilerin idlerine göre onlara ulaşmak ve onlara özel kampanyalar yapmak istersem?

In [44]:
rfm[rfm["segment"] == "cant_loose"].index

Float64Index([12346.0, 12380.0, 12482.0, 12510.0, 12891.0, 12932.0, 13044.0,
              13313.0, 13680.0, 13782.0, 13799.0, 13856.0, 14025.0, 14063.0,
              14160.0, 14221.0, 14548.0, 14607.0, 14685.0, 14745.0, 15003.0,
              15013.0, 15015.0, 15125.0, 15141.0, 15222.0, 15306.0, 15321.0,
              15359.0, 15369.0, 15372.0, 15443.0, 15538.0, 15607.0, 15633.0,
              15722.0, 15751.0, 15754.0, 15768.0, 15911.0, 15912.0, 16027.0,
              16032.0, 16158.0, 16177.0, 16197.0, 16335.0, 16467.0, 16631.0,
              16742.0, 16743.0, 16875.0, 16986.0, 17021.0, 17032.0, 17092.0,
              17113.0, 17157.0, 17188.0, 17230.0, 17268.0, 17426.0, 17448.0,
              17454.0, 17512.0, 17578.0, 17602.0, 17651.0, 17940.0, 17969.0,
              17988.0, 18009.0, 18051.0, 18064.0, 18094.0, 18251.0, 18258.0],
             dtype='float64', name='Customer ID')

In [45]:
new_df = pd.DataFrame()

In [46]:
new_df["new_customer_id"] = rfm[rfm["segment"] == "new_customers"].index

In [47]:
new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)

In [48]:
new_df.to_csv("new_customers.csv")

#### 7. Tüm Sürecin Fonksiyonlaştırılması
- eğer script olarak kullanmak istersek; aynı projede güncellemek için.

In [49]:
def create_rfm(dataframe, csv=False):

    # VERIYI HAZIRLAMA
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]

    # RFM METRIKLERININ HESAPLANMASI
    today_date = dt.datetime(2011, 12, 11)
    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                                'Invoice': lambda num: num.nunique(),
                                                "TotalPrice": lambda price: price.sum()})
    rfm.columns = ['recency', 'frequency', "monetary"]
    rfm = rfm[(rfm['monetary'] > 0)]

    # RFM SKORLARININ HESAPLANMASI
    rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

    # cltv_df skorları kategorik değere dönüştürülüp df'e eklendi
    rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                        rfm['frequency_score'].astype(str))


    # SEGMENTLERIN ISIMLENDIRILMESI
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

    rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
    rfm = rfm[["recency", "frequency", "monetary", "segment"]]
    rfm.index = rfm.index.astype(int)

    if csv:
        rfm.to_csv("rfm.csv")

    return rfm